In [1]:
import ipyparallel
import hddm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import seaborn as sns
print(hddm.__version__)

0.8.0


/home/jasongong/.local/lib/python3.7/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


# Confirmatory inference testing

In [3]:
# load the HDDM model fitted in step XX
m = hddm.load("m_full_hddm_study1_new_subj10_removed")

In [4]:
# get the MCMC chains of HDDM model
v_0V0A, v_0V1A, v_1V0A, v_1V1A, v_1Vne1A = m.nodes_db.loc[['v(0V0A)', 'v(0V1A)', 'v(1V0A)', 'v(1V1A)', 'v(1Vne1A)'], 'node']

In [26]:
# save the parameter samples for plotting
pd.DataFrame({
    "v_0V0A": v_0V0A.trace(),
    "v_0V1A":v_0V1A.trace(),
    "v_1V0A":v_1V0A.trace(),
    "v_1V1A":v_1V1A.trace(),
    "v_1Vne1A":v_1Vne1A.trace(),
             }).to_csv("plot/study_1_ddm_confirmative_analysis.csv")

In [19]:
# test the percentage of posteriors higher and lower than 0
def inference_test(trace):
    probability1 = round((trace < 0).mean(), 3) 
    probability2 = round((1-probability1), 3) 
    return "{} % < 0 < {} %".format(probability1 * 100, probability2 * 100)


### Valence hypothesis testing

In [21]:
# test 1V0A against 0
print(inference_test(v_1V0A.trace()))

# test 1V0A against 0V0A
print(inference_test(v_1V0A.trace() - v_0V0A.trace()))

100.0 % < 0 < 0.0 %
100.0 % < 0 < 0.0 %


### Arousal hypothesis testing

In [23]:
# test 0V1A against 0
print(inference_test(v_0V1A.trace()))

# test 0V1A against 0V0A
print(inference_test(v_0V1A.trace() - v_0V0A.trace()))

28.9 % < 0 < 71.1 %
30.3 % < 0 < 69.69999999999999 %


### Multi-attribute hypothesis testing

In [24]:
# compare 1V1A drift rate magnitude (converted to positive) against 0V1A drift rate magnitude
print(inference_test(-v_1V1A.trace() - v_0V1A.trace()))

# compare 1V1A drift rate magnitude (converted to positive) against 1V0A drift rate magnitude (converted to positive)
print(inference_test(-v_1V1A.trace() - -v_1V0A.trace()))

# compare 1V-1A drift rate magnitude (converted to positive) against 0V1A drift rate magnitude
print(inference_test(-v_1Vne1A.trace() - v_0V1A.trace()))

# compare 1V-1A drift rate magnitude (converted to positive) against 1V0A drift rate magnitude (converted to positive)
print(inference_test(-v_1Vne1A.trace() - -v_1V0A.trace()))


0.0 % < 0 < 100.0 %
77.2 % < 0 < 22.8 %
0.0 % < 0 < 100.0 %
42.6 % < 0 < 57.4 %


In [26]:
# compare 0V1A drift rate magnitude against 1V0A drift rate magnitude (converted to positive)
print(inference_test(-v_1V0A.trace() - v_0V1A.trace()))

0.0 % < 0 < 100.0 %


## Credible intervals for drift rate posteriors

In [150]:
# 
stats = m.gen_stats()
stats[stats.index.isin(['v(0V0A)', 'v(1V0A)', 'v(0V1A)', 'v(1V1A)', 'v(1Vne1A)'])]

,mean,std,2.5q,25q,50q,75q,97.5q,mc err
v(0V0A),-0.004064,0.024843,-0.052777,-0.020711,-0.003842,0.012387,0.044655,0.000498
v(0V1A),0.014242,0.024899,-0.034195,-0.00284,0.014728,0.030597,0.063661,0.000485
v(1V0A),-0.161679,0.025377,-0.2121,-0.178439,-0.162172,-0.144853,-0.111483,0.000568
v(1V1A),-0.134958,0.025472,-0.183634,-0.152116,-0.135537,-0.117343,-0.085592,0.000453
v(1Vne1A),-0.167925,0.024622,-0.215453,-0.184259,-0.168088,-0.151875,-0.118419,0.000492


# test the correlation between drift rate and mood valence/arosual

In [14]:
# load the correlation traces
cor_traces = pd.read_csv("study1_correlation_traces.csv")

In [22]:
# test the correlation against 0 - p values
print(inference_test(cor_traces.cor_valence))
print(inference_test(cor_traces.cor_arousal))

35.699999999999996 % < 0 < 64.3 %
39.2 % < 0 < 60.8 %


In [123]:
# get the quantiles for the correlation parameter
print(cor_traces.cor_valence.quantile(q=[0.025, 0.5, 0.975]))
print(cor_traces.cor_arousal.quantile(q=[0.025, 0.5, 0.975]))

0.025   -0.040777
0.500    0.009598
0.975    0.060154
Name: cor_valence, dtype: float64
0.025   -0.023044
0.500    0.003872
0.975    0.031421
Name: cor_arousal, dtype: float64


# Exploratory analysis

In [16]:
# Load the regression model fitted in step XX
m_full_model = hddm.load("m_full_regression_study1_new_subj_10_removed")

In [17]:
# access the sampled node for v, a, t, z
v_intercept, v_movie_valence, v_movie_arousal, v_movie_valence_movie_arousal\
= m_full_model.nodes_db.loc[["v_Intercept",
                             "v_movie_valence[T.1]",
                             "v_movie_arousal[T.1]",
                             "v_movie_valence[T.1]:movie_arousal[T.1]"], 'node']
a_intercept, a_movie_valence, a_movie_arousal, a_movie_valence_movie_arousal\
= m_full_model.nodes_db.loc[["a_Intercept",
                             "a_movie_valence[T.1]",
                             "a_movie_arousal[T.1]",
                             "a_movie_valence[T.1]:movie_arousal[T.1]"], 'node']
t_intercept, t_movie_valence, t_movie_arousal, t_movie_valence_movie_arousal\
= m_full_model.nodes_db.loc[["t_Intercept",
                             "t_movie_valence[T.1]",
                             "t_movie_arousal[T.1]",
                             "t_movie_valence[T.1]:movie_arousal[T.1]"], 'node']
z_intercept, z_movie_valence, z_movie_arousal, z_movie_valence_movie_arousal\
= m_full_model.nodes_db.loc[["z_Intercept",
                             "z_movie_valence[T.1]",
                             "z_movie_arousal[T.1]",
                             "z_movie_valence[T.1]:movie_arousal[T.1]"], 'node']

### testing regressors effects on drift rate

In [28]:
# test the regressors effects against 0
print(inference_test(v_movie_valence.trace()))
print(inference_test(v_movie_arousal.trace()))
print(inference_test(v_movie_valence_movie_arousal.trace()))

100.0 % < 0 < 0.0 %
3.4000000000000004 % < 0 < 96.6 %
87.2 % < 0 < 12.8 %


### testing regressors effects on decison boundary

In [29]:
print(inference_test(a_movie_valence.trace()))
print(inference_test(a_movie_arousal.trace()))
print(inference_test(a_movie_valence_movie_arousal.trace()))

98.8 % < 0 < 1.2 %
99.8 % < 0 < 0.2 %
16.5 % < 0 < 83.5 %


### testing regressors effects on non-decision time

In [30]:
print(inference_test(t_movie_valence.trace()))
print(inference_test(t_movie_arousal.trace()))
print(inference_test(t_movie_valence_movie_arousal.trace()))

53.2 % < 0 < 46.800000000000004 %
21.7 % < 0 < 78.3 %
31.7 % < 0 < 68.30000000000001 %


### testing regressors effects on decision bias

In [31]:
print(inference_test(z_movie_valence.trace()))
print(inference_test(z_movie_arousal.trace()))
print(inference_test(z_movie_valence_movie_arousal.trace()))

85.39999999999999 % < 0 < 14.6 %
96.6 % < 0 < 3.4000000000000004 %
5.2 % < 0 < 94.8 %


### credible intervals for parameters: v, a, t, z

In [166]:
stats = m_full_model.gen_stats()
stats[stats.index.isin(["v_Intercept",
    "v_movie_valence[T.1]",
    "v_movie_arousal[T.1]", 
    "v_movie_valence[T.1]:movie_arousal[T.1]",
    
                        "a_Intercept",
    "a_movie_valence[T.1]",
    "a_movie_arousal[T.1]", 
    "a_movie_valence[T.1]:movie_arousal[T.1]",
    
                        "t_Intercept",
    "t_movie_valence[T.1]",
    "t_movie_arousal[T.1]", 
    "t_movie_valence[T.1]:movie_arousal[T.1]",
    
                        "z_Intercept",
    "z_movie_valence[T.1]",
    "z_movie_arousal[T.1]", 
    "z_movie_valence[T.1]:movie_arousal[T.1]",])]

,mean,std,2.5q,25q,50q,75q,97.5q,mc err
v_Intercept,0.000463,0.019322,-0.037608,-0.012247,0.000312,0.013691,0.037633,0.000967
v_movie_valence[T.1],-0.143673,0.020908,-0.183477,-0.157648,-0.143708,-0.129463,-0.10247,0.001325
v_movie_arousal[T.1],0.036857,0.020527,-0.002362,0.022181,0.036771,0.050921,0.076147,0.001333
v_movie_valence[T.1]:movie_arousal[T.1],-0.031247,0.028036,-0.088474,-0.050274,-0.030411,-0.01162,0.021784,0.001855
a_Intercept,3.16628,0.061695,3.046999,3.124417,3.16699,3.206911,3.287944,0.001947
a_movie_valence[T.1],-0.083693,0.036437,-0.158431,-0.108096,-0.082877,-0.05901,-0.015306,0.002436
a_movie_arousal[T.1],-0.087047,0.034723,-0.15925,-0.109886,-0.086965,-0.063358,-0.020407,0.002362
a_movie_valence[T.1]:movie_arousal[T.1],0.045924,0.046786,-0.041671,0.013138,0.044137,0.076823,0.142416,0.003222
t_Intercept,0.943743,0.04961,0.850841,0.909737,0.942076,0.975795,1.047389,0.001427
t_movie_valence[T.1],-0.000652,0.01276,-0.024857,-0.009624,-0.00126,0.007856,0.026284,0.000884


In [167]:
# save the traces for plotting
pd.DataFrame({"v_intercept": v_intercept.trace(),
     "v_movie_valence": v_movie_valence.trace(),
     "v_movie_arousal": v_movie_arousal.trace(), 
     "v_movie_valence_movie_arousal": v_movie_valence_movie_arousal.trace(),
     "a_intercept": a_intercept.trace(),
     "a_movie_valence": a_movie_valence.trace(),
     "a_movie_arousal": a_movie_arousal.trace(),
     "a_movie_valence_movie_arousal": a_movie_valence_movie_arousal.trace(),
     "t_intercept": t_intercept.trace(), 
     "t_movie_valence": t_movie_valence.trace(),
     "t_movie_arousal": t_movie_arousal.trace(),
     "t_movie_valence_movie_arousal": t_movie_valence_movie_arousal.trace(),
     "z_intercept": z_intercept.trace(), 
     "z_movie_valence": z_movie_valence.trace(), 
     "z_movie_arousal": z_movie_arousal.trace(), 
     "z_movie_valence_movie_arousal": z_movie_valence_movie_arousal.trace()}).to_csv("plot/study_1_ddm_exploratory_analysis.csv")